# TP2: Embeddings for Scientific Domains

**Day 2 - AI for Sciences Winter School**

**Instructor:** Raphael Cousin

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/racousin/ai_for_sciences/blob/main/day2/tp2.ipynb)

## Objectives
1. Understand how different scientific domains represent data
2. Explore embeddings for text, molecules, proteins, and DNA
3. Visualize embedding spaces with dimensionality reduction (PCA, t-SNE, UMAP)
4. Use embeddings as features for classification with MLP

## Setup

Run the cell below to install and import the required packages. This may take a few minutes.

In [ ]:
# Install required packages
!pip install -q transformers datasets sentence-transformers umap-learn
!pip install -q fair-esm  # For protein embeddings (ESM-2)

import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print("Setup complete!")

---
# Part 1: Embedding Exploration

In this part, we'll explore how embeddings work across different scientific domains. For each domain:
1. **Data**: Understand the representation (text, SMILES, amino acids, nucleotides)
2. **Tokenization**: See how data is tokenized
3. **Embedding**: Extract embeddings from pre-trained models
4. **Visualization**: Use dimensionality reduction to visualize
5. **Similarity**: Find similar items using cosine similarity

---
## Section A: Scientific Text Embeddings

We'll use embeddings to analyze scientific project descriptions. The model captures semantic meaning, allowing us to find similar projects.

### The Data

Below are 15 scientific project descriptions from various fields. **Your task**: Add your own project description!

In [ ]:
# 15 Scientific project descriptions from different fields
projects = [
    # Physics / Astrophysics
    "Detecting gravitational waves from binary black hole mergers using deep learning on LIGO data",
    "Simulating dark matter distribution in galaxy clusters with N-body simulations",
    "Quantum error correction codes for fault-tolerant quantum computing",
    
    # Biology / Medicine
    "Predicting protein-protein interactions using graph neural networks",
    "Single-cell RNA sequencing analysis to identify cancer cell subtypes",
    "Drug discovery for Alzheimer's disease targeting amyloid-beta aggregation",
    
    # Chemistry / Materials
    "Machine learning for predicting catalyst activity in CO2 reduction reactions",
    "Designing new battery materials using high-throughput DFT calculations",
    "Molecular dynamics simulation of polymer degradation in marine environments",
    
    # Environmental / Climate
    "Climate model downscaling using convolutional neural networks",
    "Predicting extreme weather events from satellite imagery with transformers",
    "Ocean acidification impact on coral reef ecosystems modeling",
    
    # Computer Science / AI
    "Natural language processing for scientific literature summarization",
    "Reinforcement learning for autonomous drone navigation in complex environments",
    "Federated learning for privacy-preserving medical image analysis"
]

# Field labels for coloring
fields = [
    "Physics", "Physics", "Physics",
    "Biology", "Biology", "Biology",
    "Chemistry", "Chemistry", "Chemistry",
    "Climate", "Climate", "Climate",
    "Computer Science", "Computer Science", "Computer Science"
]

print(f"Number of projects: {len(projects)}")
print(f"\nFields: {set(fields)}")
print(f"\nExample project (Biology):")
print(f"  '{projects[3]}'")

### Exercise: Add Your Own Project

Add your own project description to the list. What field does it belong to?

In [ ]:
# TODO: Add your own project!
my_project = "Your project description here"  # <-- Modify this!
my_field = "Your field"  # <-- Modify this! (Physics, Biology, Chemistry, Climate, Computer Science, or other)

# Add to lists
projects_with_yours = projects + [my_project]
fields_with_yours = fields + [my_field]

print(f"Total projects: {len(projects_with_yours)}")
print(f"Your project: '{my_project}'")
print(f"Your field: {my_field}")

### Load Text Embedding Model

We'll use **Sentence-BERT** (`all-MiniLM-L6-v2`), a model specifically trained to produce meaningful sentence embeddings.

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the model
text_model = SentenceTransformer('all-MiniLM-L6-v2')

print(f"Model: all-MiniLM-L6-v2")
print(f"Embedding dimension: {text_model.get_sentence_embedding_dimension()}")
print(f"Max sequence length: {text_model.max_seq_length}")

### Tokenization

Let's see how the model tokenizes text:

In [ ]:
# Get the tokenizer from the model
tokenizer = text_model.tokenizer

# Example tokenization
example_text = projects[3]  # Biology project
tokens = tokenizer.tokenize(example_text)

print(f"Original text:")
print(f"  '{example_text}'")
print(f"\nTokens ({len(tokens)} tokens):")
print(f"  {tokens}")
print(f"\nNote: '##' prefix indicates subword continuation (WordPiece tokenization)")

### Generate Embeddings

In [ ]:
# Generate embeddings for all projects
text_embeddings = text_model.encode(projects_with_yours, show_progress_bar=True)

print(f"Embeddings shape: {text_embeddings.shape}")
print(f"  - {text_embeddings.shape[0]} projects")
print(f"  - {text_embeddings.shape[1]} dimensions per embedding")

### Visualize with PCA/t-SNE

Since we have a small dataset (16 points), we'll use **PCA** or **t-SNE** for visualization. UMAP works better with larger datasets.

In [ ]:
# Reduce to 2D using PCA
pca = PCA(n_components=2)
text_2d_pca = pca.fit_transform(text_embeddings)

# Also try t-SNE
tsne = TSNE(n_components=2, perplexity=5, random_state=42)  # Low perplexity for small dataset
text_2d_tsne = tsne.fit_transform(text_embeddings)

# Create color mapping
unique_fields = list(set(fields_with_yours))
colors = plt.cm.tab10(np.linspace(0, 1, len(unique_fields)))
field_to_color = {field: colors[i] for i, field in enumerate(unique_fields)}
point_colors = [field_to_color[f] for f in fields_with_yours]

# Plot both
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PCA plot
for field in unique_fields:
    mask = [f == field for f in fields_with_yours]
    axes[0].scatter(text_2d_pca[mask, 0], text_2d_pca[mask, 1], 
                    c=[field_to_color[field]], label=field, s=100, alpha=0.7)
axes[0].set_title('PCA Visualization')
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')
axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0].grid(True, alpha=0.3)

# t-SNE plot
for field in unique_fields:
    mask = [f == field for f in fields_with_yours]
    axes[1].scatter(text_2d_tsne[mask, 0], text_2d_tsne[mask, 1], 
                    c=[field_to_color[field]], label=field, s=100, alpha=0.7)
axes[1].set_title('t-SNE Visualization')
axes[1].set_xlabel('t-SNE 1')
axes[1].set_ylabel('t-SNE 2')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nQuestion: Do projects from the same field cluster together?")
print("Question: Where does your project appear?")

### Similarity Search

Find the most similar projects using cosine similarity:

In [ ]:
# Compute similarity matrix
similarity_matrix = cosine_similarity(text_embeddings)

# Find most similar project to YOUR project (last one)
your_similarities = similarity_matrix[-1, :-1]  # Exclude self-similarity
most_similar_idx = np.argmax(your_similarities)

print("Your project:")
print(f"  '{projects_with_yours[-1]}'")
print(f"\nMost similar project (similarity: {your_similarities[most_similar_idx]:.3f}):")
print(f"  '{projects[most_similar_idx]}'")
print(f"  Field: {fields[most_similar_idx]}")

print(f"\nTop 3 similar projects:")
top_3_idx = np.argsort(your_similarities)[-3:][::-1]
for i, idx in enumerate(top_3_idx):
    print(f"  {i+1}. [{fields[idx]}] {projects[idx][:60]}... (sim: {your_similarities[idx]:.3f})")

### Optional: Try a Different Model

You can try **SciBERT** (trained on scientific papers) to see if it gives different results.

In [ ]:
# Optional: Try SciBERT
# Uncomment the lines below to try a different model

# from transformers import AutoTokenizer, AutoModel
# 
# scibert_tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
# scibert_model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')
# 
# # Get embeddings (using [CLS] token)
# def get_scibert_embedding(text):
#     inputs = scibert_tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
#     with torch.no_grad():
#         outputs = scibert_model(**inputs)
#     return outputs.last_hidden_state[:, 0, :].numpy()  # [CLS] token
# 
# scibert_embeddings = np.vstack([get_scibert_embedding(p) for p in projects_with_yours])
# print(f"SciBERT embeddings shape: {scibert_embeddings.shape}")

print("Uncomment the code above to try SciBERT!")

---
## Section B: Molecular Embeddings (Chemistry)

Molecules are represented as **SMILES** (Simplified Molecular Input Line Entry System) strings. These are text representations of molecular structure.

### Understanding SMILES

| Symbol | Meaning |
|--------|--------|
| C, N, O, S | Atoms (carbon, nitrogen, oxygen, sulfur) |
| c, n, o | Aromatic atoms (lowercase) |
| = | Double bond |
| # | Triple bond |
| () | Branch |
| [] | Explicit atom properties |
| 1,2,3... | Ring closures |

In [ ]:
# Example molecules with their SMILES and names
molecules = {
    # Simple molecules
    "Water": "O",
    "Methane": "C",
    "Ethanol": "CCO",
    "Acetic acid": "CC(=O)O",
    
    # Drugs - Pain/Inflammation
    "Aspirin": "CC(=O)Oc1ccccc1C(=O)O",
    "Ibuprofen": "CC(C)Cc1ccc(cc1)C(C)C(=O)O",
    "Acetaminophen": "CC(=O)Nc1ccc(O)cc1",
    
    # Drugs - Antibiotics
    "Penicillin G": "CC1(C)SC2C(NC(=O)Cc3ccccc3)C(=O)N2C1C(=O)O",
    "Amoxicillin": "CC1(C)SC2C(NC(=O)C(N)c3ccc(O)cc3)C(=O)N2C1C(=O)O",
    
    # Drugs - CNS
    "Caffeine": "Cn1cnc2c1c(=O)n(c(=O)n2C)C",
    "Diazepam": "CN1C(=O)CN=C(c2ccccc2)c3cc(Cl)ccc13",
    
    # Drugs - Cardiovascular
    "Atorvastatin": "CC(C)c1c(C(=O)Nc2ccccc2)c(c(c3ccc(F)cc3)n1CC(O)CC(O)CC(=O)O)c4ccccc4",
    "Lisinopril": "NCCCC[C@H](N[C@@H](CCc1ccccc1)C(=O)O)C(=O)N2CCC[C@H]2C(=O)O",
}

# Drug categories
drug_categories = {
    "Water": "Simple", "Methane": "Simple", "Ethanol": "Simple", "Acetic acid": "Simple",
    "Aspirin": "Pain", "Ibuprofen": "Pain", "Acetaminophen": "Pain",
    "Penicillin G": "Antibiotic", "Amoxicillin": "Antibiotic",
    "Caffeine": "CNS", "Diazepam": "CNS",
    "Atorvastatin": "Cardiovascular", "Lisinopril": "Cardiovascular",
}

print("Example SMILES representations:")
for name, smiles in list(molecules.items())[:5]:
    print(f"  {name:15s}: {smiles}")

print(f"\nTotal molecules: {len(molecules)}")
print(f"Categories: {set(drug_categories.values())}")

### Load ChemBERTa Model

**ChemBERTa** is a transformer model trained on SMILES strings. It understands molecular structure.

In [ ]:
from transformers import AutoTokenizer, AutoModel

# Load ChemBERTa
chem_tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
chem_model = AutoModel.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
chem_model.eval()

print("Model: ChemBERTa-zinc-base-v1")
print(f"Vocabulary size: {chem_tokenizer.vocab_size}")
print(f"Embedding dimension: {chem_model.config.hidden_size}")

### SMILES Tokenization

See how ChemBERTa tokenizes SMILES strings:

In [ ]:
# Tokenize aspirin
aspirin_smiles = molecules["Aspirin"]
tokens = chem_tokenizer.tokenize(aspirin_smiles)

print(f"Molecule: Aspirin")
print(f"SMILES: {aspirin_smiles}")
print(f"\nTokens ({len(tokens)}):")
print(f"  {tokens}")
print(f"\nNote: The tokenizer learns subword units from SMILES patterns")

### Generate Molecular Embeddings

In [ ]:
def get_molecule_embedding(smiles, tokenizer, model):
    """Get embedding for a SMILES string using [CLS] token."""
    inputs = tokenizer(smiles, return_tensors='pt', truncation=True, max_length=512, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use [CLS] token embedding (first token)
    return outputs.last_hidden_state[:, 0, :].numpy().flatten()

# Generate embeddings for all molecules
mol_names = list(molecules.keys())
mol_smiles = list(molecules.values())
mol_categories = [drug_categories[name] for name in mol_names]

mol_embeddings = np.array([get_molecule_embedding(s, chem_tokenizer, chem_model) for s in mol_smiles])

print(f"Molecular embeddings shape: {mol_embeddings.shape}")
print(f"  - {mol_embeddings.shape[0]} molecules")
print(f"  - {mol_embeddings.shape[1]} dimensions")

### Visualize with UMAP

In [ ]:
import umap

# Reduce to 2D with UMAP
reducer = umap.UMAP(n_neighbors=5, min_dist=0.3, random_state=42)
mol_2d = reducer.fit_transform(mol_embeddings)

# Create plot
fig, ax = plt.subplots(figsize=(10, 8))

# Color by category
unique_categories = list(set(mol_categories))
colors = plt.cm.Set2(np.linspace(0, 1, len(unique_categories)))
cat_to_color = {cat: colors[i] for i, cat in enumerate(unique_categories)}

for cat in unique_categories:
    mask = [c == cat for c in mol_categories]
    ax.scatter(mol_2d[mask, 0], mol_2d[mask, 1], 
               c=[cat_to_color[cat]], label=cat, s=150, alpha=0.7)

# Add labels
for i, name in enumerate(mol_names):
    ax.annotate(name, (mol_2d[i, 0], mol_2d[i, 1]), fontsize=8, alpha=0.8)

ax.set_title('Molecular Embeddings (ChemBERTa + UMAP)', fontsize=14)
ax.set_xlabel('UMAP 1')
ax.set_ylabel('UMAP 2')
ax.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nQuestion: Do similar drugs (same category) cluster together?")
print("Question: What do you notice about the pain relievers?")

### Molecular Similarity Search

In [ ]:
# Find most similar molecule to Aspirin
query_mol = "Aspirin"
query_idx = mol_names.index(query_mol)
query_embedding = mol_embeddings[query_idx]

# Compute similarities
similarities = cosine_similarity([query_embedding], mol_embeddings)[0]
similarities[query_idx] = -1  # Exclude self

# Top 3 similar
top_3_idx = np.argsort(similarities)[-3:][::-1]

print(f"Query molecule: {query_mol}")
print(f"  SMILES: {molecules[query_mol]}")
print(f"  Category: {drug_categories[query_mol]}")

print(f"\nMost similar molecules:")
for i, idx in enumerate(top_3_idx):
    name = mol_names[idx]
    print(f"  {i+1}. {name} ({drug_categories[name]}) - similarity: {similarities[idx]:.3f}")

---
## Section C: Protein Embeddings (Biology)

Proteins are sequences of **amino acids** (20 standard types). Each amino acid is represented by a single letter.

| Letter | Amino Acid | Property |
|--------|------------|----------|
| A | Alanine | Hydrophobic |
| C | Cysteine | Forms disulfide bonds |
| D | Aspartic acid | Negative charge |
| E | Glutamic acid | Negative charge |
| K | Lysine | Positive charge |
| R | Arginine | Positive charge |
| ... | ... | ... |

In [ ]:
# Example protein sequences from different families
proteins = {
    # Hemoglobin family (oxygen transport)
    "Hemoglobin_alpha": "MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSH",
    "Hemoglobin_beta": "MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLST",
    "Myoglobin": "MGLSDGEWQLVLNVWGKVEADIPGHGQEVLIRLFKGHPETLEKFDKFKHLK",
    
    # Kinase family (phosphorylation)
    "PKA_catalytic": "MGNAAAAKKGSEQESVKEFLAKAKEDFLKKWENPAQNTAHLDQFERIKTLG",
    "PKC_alpha": "MADVFPGNDSASGPGNTSTGSADPSSAPGSEHCGSAGSGSPTGSPGSSGGP",
    "CDK2": "MENFQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPSTAIRE",
    
    # Protease family (protein cleavage)
    "Trypsin": "IVGGYTCGANTVPYQVSLNSGYHFCGGSLINSQWVVSAAHCYKSGIQVRLG",
    "Chymotrypsin": "CGVPAIQPVLSGLSRIVNGEEAVPGSWPWQVSLQDKTGFHFCGGSLINENWVVTAAHCGVTTSDVVVAGEFDQGSSSEKIQKLKIAKVFKNS",
    "Elastase": "VVGGTEAQRNSWPSQISLQYRSGSSWAHTCGGTLIRQNWVMTAAHCVDRELTFRVVVGEHNLNQNDGTEQYVGVQKIVVHPYWNTDDVAAGYDIALLRLAQSVTLNSYVQLGVLPRAGTILANNSPCYITGWGLTRTNGQLAQTLQQAYLPTVDYAICSSSSYWGSTVKNSMVCAGGDGVRSGCQGDSGGPLHCLVNGQYAVHGVTSFVSRLGCNVTRKPTVFTRVSAYISWINNVIASN",
    
    # Transporter family
    "GLUT1": "MEPSSKKLTGRLMLAVGGAVLGSLQFGYNTGVINAPQKVIEEFYNQTWVHR",
    "Aquaporin1": "MASEFKKKLFWRAVVAEFLATTLFVFISIGSALGFKYPVGNNQTAVQDNVKVSLAFGLSIATLAQSVGHISGAHLNPAVTLGLLLSCQISIFRALMYIIAQCVGAIVATAILSGITSSLTGNSLGRNDLADGVNSGQGLGIEIIGTLQLVLCVLATTDRRRRDLGGSAPLAIGLSVALGHLLAIDYTGCGINPARSFGSAVITHNFSNHWIFWVGPFIGGALAGLIYDFLLFPRLKSISV",
}

protein_families = {
    "Hemoglobin_alpha": "Globin", "Hemoglobin_beta": "Globin", "Myoglobin": "Globin",
    "PKA_catalytic": "Kinase", "PKC_alpha": "Kinase", "CDK2": "Kinase",
    "Trypsin": "Protease", "Chymotrypsin": "Protease", "Elastase": "Protease",
    "GLUT1": "Transporter", "Aquaporin1": "Transporter",
}

print("Example protein sequences (first 50 amino acids):")
for name, seq in list(proteins.items())[:3]:
    print(f"  {name:20s}: {seq[:50]}...")

print(f"\nTotal proteins: {len(proteins)}")
print(f"Families: {set(protein_families.values())}")

### Load ESM-2 Model

**ESM-2** (Evolutionary Scale Modeling) is a state-of-the-art protein language model from Meta AI.

In [ ]:
import esm

# Load ESM-2 (small version for speed)
esm_model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()  # 8M parameter version
batch_converter = alphabet.get_batch_converter()
esm_model.eval()

print("Model: ESM-2 (esm2_t6_8M_UR50D)")
print(f"Parameters: 8 million")
print(f"Embedding dimension: {esm_model.embed_dim}")

### Protein Tokenization

Proteins are tokenized at the amino acid level (each letter = one token):

In [ ]:
# Show tokenization
example_seq = proteins["Hemoglobin_alpha"][:20]
print(f"Sequence: {example_seq}")
print(f"\nTokens (each amino acid is a token):")
print(f"  {list(example_seq)}")
print(f"\nUnlike text, protein tokenization is typically character-level (one amino acid = one token)")

### Generate Protein Embeddings

In [ ]:
def get_protein_embedding(name, sequence, model, batch_converter):
    """Get mean embedding for a protein sequence."""
    data = [(name, sequence)]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[6])  # Last layer
    
    # Get mean embedding (excluding special tokens)
    token_embeddings = results["representations"][6]
    # Mean over sequence length (excluding BOS/EOS)
    mean_embedding = token_embeddings[0, 1:-1, :].mean(dim=0).numpy()
    return mean_embedding

# Generate embeddings
protein_names = list(proteins.keys())
protein_seqs = list(proteins.values())
protein_fams = [protein_families[name] for name in protein_names]

print("Generating protein embeddings...")
protein_embeddings = np.array([
    get_protein_embedding(name, seq, esm_model, batch_converter) 
    for name, seq in zip(protein_names, protein_seqs)
])

print(f"Protein embeddings shape: {protein_embeddings.shape}")

### Visualize Protein Embeddings

In [ ]:
# UMAP visualization
reducer = umap.UMAP(n_neighbors=3, min_dist=0.1, random_state=42)
protein_2d = reducer.fit_transform(protein_embeddings)

# Plot
fig, ax = plt.subplots(figsize=(10, 8))

unique_fams = list(set(protein_fams))
colors = plt.cm.Set1(np.linspace(0, 1, len(unique_fams)))
fam_to_color = {fam: colors[i] for i, fam in enumerate(unique_fams)}

for fam in unique_fams:
    mask = [f == fam for f in protein_fams]
    ax.scatter(protein_2d[mask, 0], protein_2d[mask, 1],
               c=[fam_to_color[fam]], label=fam, s=150, alpha=0.7)

# Add labels
for i, name in enumerate(protein_names):
    ax.annotate(name, (protein_2d[i, 0], protein_2d[i, 1]), fontsize=8, alpha=0.8)

ax.set_title('Protein Embeddings (ESM-2 + UMAP)', fontsize=14)
ax.set_xlabel('UMAP 1')
ax.set_ylabel('UMAP 2')
ax.legend(title='Family', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nQuestion: Do proteins from the same family cluster together?")
print("Question: Which families are most similar to each other?")

---
## Section D: DNA Embeddings (Genomics)

DNA is a sequence of 4 nucleotides: **A** (Adenine), **T** (Thymine), **G** (Guanine), **C** (Cytosine).

### K-mer Tokenization

DNA models often use **k-mers** (subsequences of length k) as tokens:
- Sequence: `ATCGATCG`
- 3-mers: `ATC`, `TCG`, `CGA`, `GAT`, `ATC`, `TCG`

In [ ]:
# Example DNA sequences (promoter regions)
dna_sequences = {
    # Housekeeping gene promoters (always active)
    "GAPDH_promoter": "GCCCGCCCCCGCCCGCCCGCCGCCGCCGCCGCCGCCGCCGCCCGCCCGCGCGCCCGCCATGGGGAAGGTGAAGGTCGGAGTCAACGGATTTGGTCGTAT",
    "Actin_promoter": "CCCCCGCCCCGCGCCCCGCCCCGCGCCCCGCCCCGCGCCCCGCCCCGCGCCGCGCGCCATGGATGATGATATCGCCGCGCTCGTCGTCGACAACGGCTC",
    "Tubulin_promoter": "GCGCCGCCCCCGCCGCCGCCGCCGCCGCCGCCGCGCGCGCGCGCGCGCGCGCGCGCATGCGTGAGTGCATCTCCATCCACGTTGGCCAGGCTGGTGTC",
    
    # Immune response promoters
    "IL6_promoter": "AATAAATAAATAAATAAATGCCCCTCAGCTTGACTCACCTGAGACGTGCAGAGCTGGCAGAAGAAAGCAGCAAAGAGGCACTGGCAGAAAACAACCT",
    "TNF_promoter": "AAAAAATATTTATATATTTATATATATGCGCGCCGCGCCGCGCCGCGCCGCGCCGCAGGCAGGCAGGCAGGCAGGCAGGCTGAGCTGAGCTGAGCTGA",
    "IFN_promoter": "AATAAAATAAATTTTTTATAATTTTAATTTTAATTTTAAATTGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGC",
    
    # Cell cycle promoters
    "CyclinD_promoter": "GCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCATGAACTACCTGGACCGCTTCCTGTCGCTGGAGCCCGTGAA",
    "CDK1_promoter": "CCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCGCCATGGAAGATTATACCAAAATAGAGAAAATTGGAGAAGGTAC",
    "p53_promoter": "ATATATATATATATATATATATAGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCATGGAGGAGCCGCAGTCAGATCCTAGCGTCGAG",
}

dna_categories = {
    "GAPDH_promoter": "Housekeeping", "Actin_promoter": "Housekeeping", "Tubulin_promoter": "Housekeeping",
    "IL6_promoter": "Immune", "TNF_promoter": "Immune", "IFN_promoter": "Immune",
    "CyclinD_promoter": "Cell_cycle", "CDK1_promoter": "Cell_cycle", "p53_promoter": "Cell_cycle",
}

print("Example DNA sequences (first 50 nucleotides):")
for name, seq in list(dna_sequences.items())[:3]:
    print(f"  {name:18s}: {seq[:50]}...")

print(f"\nTotal sequences: {len(dna_sequences)}")
print(f"Categories: {set(dna_categories.values())}")

### K-mer Tokenization Example

In [ ]:
def get_kmers(sequence, k=6):
    """Convert sequence to k-mers."""
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

# Example
example_dna = "ATCGATCGATCG"
kmers_3 = get_kmers(example_dna, k=3)
kmers_6 = get_kmers(example_dna, k=6)

print(f"Sequence: {example_dna}")
print(f"\n3-mers: {kmers_3}")
print(f"\n6-mers: {kmers_6}")
print(f"\nDNABERT uses 6-mers (4^6 = 4096 possible tokens)")

### Load DNABERT Model

In [ ]:
from transformers import AutoTokenizer, AutoModel

# Load DNABERT-2 (or fallback to DNABERT)
try:
    dna_tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
    dna_model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
    print("Model: DNABERT-2-117M")
except:
    # Fallback: use a simpler DNA model or generic BERT with k-mers
    print("DNABERT-2 not available, using fallback approach...")
    dna_tokenizer = None
    dna_model = None

if dna_model is not None:
    dna_model.eval()
    print(f"Embedding dimension: {dna_model.config.hidden_size}")

### Generate DNA Embeddings

In [ ]:
def get_dna_embedding(sequence, tokenizer, model):
    """Get embedding for a DNA sequence."""
    if tokenizer is None or model is None:
        # Fallback: use simple k-mer frequency vector
        from collections import Counter
        kmers = get_kmers(sequence, k=4)
        counts = Counter(kmers)
        # Create a fixed-size vector based on common 4-mers
        all_4mers = [''.join(p) for p in __import__('itertools').product('ACGT', repeat=4)]
        return np.array([counts.get(kmer, 0) for kmer in all_4mers], dtype=float)
    
    inputs = tokenizer(sequence, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy().flatten()

# Generate embeddings
dna_names = list(dna_sequences.keys())
dna_seqs = list(dna_sequences.values())
dna_cats = [dna_categories[name] for name in dna_names]

print("Generating DNA embeddings...")
dna_embeddings = np.array([get_dna_embedding(seq, dna_tokenizer, dna_model) for seq in dna_seqs])

print(f"DNA embeddings shape: {dna_embeddings.shape}")

### Visualize DNA Embeddings

In [ ]:
# UMAP visualization
reducer = umap.UMAP(n_neighbors=3, min_dist=0.1, random_state=42)
dna_2d = reducer.fit_transform(dna_embeddings)

# Plot
fig, ax = plt.subplots(figsize=(10, 8))

unique_cats = list(set(dna_cats))
colors = plt.cm.Set1(np.linspace(0, 1, len(unique_cats)))
cat_to_color = {cat: colors[i] for i, cat in enumerate(unique_cats)}

for cat in unique_cats:
    mask = [c == cat for c in dna_cats]
    ax.scatter(dna_2d[mask, 0], dna_2d[mask, 1],
               c=[cat_to_color[cat]], label=cat, s=150, alpha=0.7)

# Add labels
for i, name in enumerate(dna_names):
    short_name = name.replace('_promoter', '')
    ax.annotate(short_name, (dna_2d[i, 0], dna_2d[i, 1]), fontsize=8, alpha=0.8)

ax.set_title('DNA Embeddings (DNABERT + UMAP)', fontsize=14)
ax.set_xlabel('UMAP 1')
ax.set_ylabel('UMAP 2')
ax.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nQuestion: Do promoters with similar functions cluster together?")
print("Question: What makes housekeeping genes different from immune genes?")

---
## Part 1 Summary: Comparing Domains

| Domain | Data Format | Tokenization | Model | Vocabulary |
|--------|-------------|--------------|-------|------------|
| Text | Natural language | WordPiece/BPE | Sentence-BERT | ~30k tokens |
| Molecules | SMILES strings | SMILES tokenizer | ChemBERTa | ~600 tokens |
| Proteins | Amino acid sequence | Per-residue | ESM-2 | 20 amino acids |
| DNA | Nucleotide sequence | k-mers | DNABERT | 4 nucleotides (4^k k-mers) |

---
# Part 2: Classification with Embeddings + MLP

Now we'll use molecular embeddings as features for a real classification task: **predicting Blood-Brain Barrier Penetration (BBBP)**.

This is a critical property for CNS drug development - can the drug cross the blood-brain barrier to reach the brain?

## Load the BBBP Dataset

In [ ]:
import pandas as pd

# Load BBBP dataset from MoleculeNet (via DeepChem or direct URL)
bbbp_url = "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/BBBP.csv"

try:
    bbbp_df = pd.read_csv(bbbp_url)
    print(f"Loaded BBBP dataset: {len(bbbp_df)} molecules")
except:
    print("Could not load from URL, creating sample dataset...")
    # Create a sample dataset
    sample_data = {
        'smiles': [
            'CC(C)Cc1ccc(cc1)C(C)C(=O)O',  # Ibuprofen - penetrates
            'CC(=O)Nc1ccc(O)cc1',  # Acetaminophen - penetrates
            'CC(=O)Oc1ccccc1C(=O)O',  # Aspirin - penetrates
            'Cn1cnc2c1c(=O)n(c(=O)n2C)C',  # Caffeine - penetrates
            'CC1(C)SC2C(NC(=O)Cc3ccccc3)C(=O)N2C1C(=O)O',  # Penicillin - does not penetrate
        ] * 40,
        'p_np': [1, 1, 1, 1, 0] * 40
    }
    bbbp_df = pd.DataFrame(sample_data)

print(f"\nDataset shape: {bbbp_df.shape}")
print(f"\nColumns: {bbbp_df.columns.tolist()}")
print(f"\nLabel distribution:")
print(bbbp_df['p_np'].value_counts())
print("  1 = Penetrates BBB")
print("  0 = Does not penetrate")

## Generate Embeddings for BBBP Dataset

We'll use a subset for speed:

In [ ]:
# Use a subset for speed (full dataset would take too long)
n_samples = min(200, len(bbbp_df))
bbbp_subset = bbbp_df.sample(n=n_samples, random_state=42).reset_index(drop=True)

print(f"Using {n_samples} molecules for classification")
print(f"Label distribution in subset:")
print(bbbp_subset['p_np'].value_counts())

# Generate embeddings
print("\nGenerating embeddings (this may take a minute)...")
bbbp_embeddings = []
valid_indices = []

for i, smiles in enumerate(bbbp_subset['smiles']):
    try:
        emb = get_molecule_embedding(smiles, chem_tokenizer, chem_model)
        bbbp_embeddings.append(emb)
        valid_indices.append(i)
    except:
        continue  # Skip invalid SMILES
    
    if (i + 1) % 50 == 0:
        print(f"  Processed {i + 1}/{n_samples}")

bbbp_embeddings = np.array(bbbp_embeddings)
bbbp_labels = bbbp_subset.iloc[valid_indices]['p_np'].values

print(f"\nFinal embeddings shape: {bbbp_embeddings.shape}")
print(f"Labels shape: {bbbp_labels.shape}")

## Split Data and Prepare for PyTorch

In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    bbbp_embeddings, bbbp_labels, test_size=0.2, random_state=42, stratify=bbbp_labels
)

# Convert to PyTorch tensors
X_train_t = torch.FloatTensor(X_train)
X_test_t = torch.FloatTensor(X_test)
y_train_t = torch.FloatTensor(y_train).unsqueeze(1)
y_test_t = torch.FloatTensor(y_test).unsqueeze(1)

print(f"Training set: {X_train_t.shape[0]} samples")
print(f"Test set: {X_test_t.shape[0]} samples")
print(f"Input dimension: {X_train_t.shape[1]}")

## Build MLP Classifier

In [ ]:
class MLPClassifier(nn.Module):
    """Simple MLP for binary classification on embeddings."""
    
    def __init__(self, input_dim, hidden_dim=128):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.network(x)

# Create model
input_dim = X_train_t.shape[1]
model = MLPClassifier(input_dim=input_dim, hidden_dim=128)

print(f"Model architecture:")
print(model)

# Count parameters
n_params = sum(p.numel() for p in model.parameters())
print(f"\nTotal parameters: {n_params:,}")

## Train the Classifier

In [ ]:
# Training setup
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
n_epochs = 100
train_losses = []
test_losses = []

for epoch in range(n_epochs):
    # Training
    model.train()
    optimizer.zero_grad()
    y_pred_train = model(X_train_t)
    loss = criterion(y_pred_train, y_train_t)
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())
    
    # Evaluation
    model.eval()
    with torch.no_grad():
        y_pred_test = model(X_test_t)
        test_loss = criterion(y_pred_test, y_test_t)
        test_losses.append(test_loss.item())
    
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1:3d}: Train Loss = {loss.item():.4f}, Test Loss = {test_loss.item():.4f}")

print("\nTraining complete!")

## Evaluate the Model

In [ ]:
# Plot training curves
plt.figure(figsize=(10, 4))
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Final evaluation
model.eval()
with torch.no_grad():
    y_pred_proba = model(X_test_t).numpy()
    y_pred = (y_pred_proba > 0.5).astype(int)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"\n" + "="*50)
print(f" CLASSIFICATION RESULTS")
print(f"="*50)
print(f"\nTest Accuracy: {accuracy:.2%}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['No penetration', 'Penetrates BBB']))

## Exercise: Experiment with the MLP

Try modifying the classifier to improve performance:

In [ ]:
# TODO: Experiment with different architectures
# Try changing:
# - hidden_dim: 64, 128, 256
# - Number of layers
# - Dropout rate
# - Learning rate

class ImprovedMLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=256):  # <-- Modify hidden_dim!
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),  # <-- Modify dropout!
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            # Add more layers here if you want
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.network(x)

# Train and evaluate your improved model
# model_v2 = ImprovedMLPClassifier(input_dim)
# ... (training code)

print("Modify the class above and train your improved model!")

---
# Summary

In this practical, you learned:

## Part 1: Embedding Exploration
1. **Text embeddings**: Sentence-BERT captures semantic meaning of scientific projects
2. **Molecular embeddings**: ChemBERTa encodes chemical structure from SMILES
3. **Protein embeddings**: ESM-2 captures evolutionary and structural information
4. **DNA embeddings**: DNABERT understands genomic sequences via k-mers

## Part 2: Classification
5. **Embeddings as features**: Pre-trained embeddings can be used as input to downstream tasks
6. **MLP classifier**: Simple neural network on top of embeddings for classification
7. **BBBP prediction**: Real-world drug discovery application

## Key Takeaways
- Different domains require different tokenization strategies
- Pre-trained embeddings capture domain-specific knowledge
- Visualization (PCA, t-SNE, UMAP) helps understand embedding spaces
- Embeddings enable transfer learning for downstream tasks